In [209]:
from itertools import combinations
from itertools import permutations

In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("yarn").setAppName("Lab 4")
sc = SparkContext.getOrCreate(conf=conf)

In [3]:
sc

<SparkContext master=yarn appName=Lab 4>

In [4]:
data = sc.textFile('retail.dat')
data.collect()

['30 31 32 ',
 '33 34 35 ',
 '36 37 38 39 40 41 42 43 44 45 46 ',
 '38 39 47 48 ',
 '38 39 48 49 50 51 52 53 54 55 56 57 58 ',
 '32 41 59 60 61 62 ',
 '3 39 48 ',
 '63 64 65 66 67 68 ',
 '32 69 ',
 '48 70 71 72 ',
 '39 73 74 75 76 77 78 79 ',
 '36 38 39 41 48 79 80 81 ',
 '82 83 84 ',
 '41 85 86 87 88 ',
 '39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 ',
 '36 38 39 48 89 ',
 '39 41 102 103 104 105 106 107 108 ',
 '38 39 41 109 110 ',
 '39 111 112 113 114 115 116 117 118 ',
 '119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 ',
 '48 134 135 136 ',
 '39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 ',
 '39 150 151 152 ',
 '38 39 56 153 154 155 ',
 '48 156 157 158 159 160 ',
 '39 41 48 ',
 '161 162 163 164 165 166 167 ',
 '38 39 48 168 169 170 171 172 173 ',
 '32 39 41 48 174 175 176 177 178 ',
 '32 38 39 47 48 179 180 181 182 183 ',
 '39 184 185 186 ',
 '36 38 41 48 140 187 188 ',
 '39 48 186 189 190 191 192 193 194 195 196 197 198 199 200 ',
 '39 201 202 203 204 205 

In [300]:
def setOfCombination(items, k):
    # to hop chap k cua n items
    
    list_of_items = sorted(items.rsplit(), key=int)
    result = list(permutations(list_of_items, k))
    return result

# ---------------------------------------------------------------------------
def frequentItemsets(data, k = 1):
    result = 0
    if k == 1:
        itemsets_data = data.flatMap(lambda x: x.rsplit()).map(lambda x: (x,))
    else:
        itemsets_data = data.flatMap(lambda x: setOfCombination(x, k))

    result = itemsets_data.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y).filter(lambda x: x[1] > 1)
    return result;

# ---------------------------------------------------------------------------
def getSupport(items):
    # return frequent_itemsets.filter(lambda x: x[0] == items).collect()[0][1]

    result = frequent_itemsets.filter(lambda x: x[0] == items)
    return result.collect()[0][1]

# ---------------------------------------------------------------------------
def getConfidence(item_tuple1, item_tuple2):
    # confidence of (item_tuple1 --> item_tuple2)
    
    support_item1_item2 = getSupport(item_tuple1 + item_tuple2)
    support_item1 = getSupport(item_tuple1)
    
    return support_item1_item2/support_item1
# ---------------------------------------------------------------------------
def getConfidenceFromResult(rdd, from_, to_):
    confidence = rdd.filter(lambda x: x[0] == from_ and x[1] == to_)
    if confidence.isEmpty():
        return 'this association rule does not exist'
    else:
        return confidence.collect()

In [311]:
# I am so sorry for hardcode this step due to the limitation of time
# according to my friend's result, set of frequent-itemsets with 5 items each sets does not exist

frequent_itemsets = frequentItemsets(data, 1)\
                    .union(frequentItemsets(data, 2))\
                    .union(frequentItemsets(data, 3))\
                    .union(frequentItemsets(data, 4))
sorted(frequent_itemsets.collect(), key=lambda x: int(x[0][0]))

[(('32',), 5),
 (('32', '41'), 2),
 (('32', '39'), 2),
 (('32', '48'), 2),
 (('32', '39', '48'), 2),
 (('32', '48', '39'), 2),
 (('36',), 6),
 (('36', '38'), 6),
 (('36', '39'), 5),
 (('36', '41'), 3),
 (('36', '38', '39'), 5),
 (('36', '38', '41'), 3),
 (('36', '39', '38'), 5),
 (('36', '39', '41'), 2),
 (('36', '41', '38'), 3),
 (('36', '41', '39'), 2),
 (('36', '38', '39', '41'), 2),
 (('36', '38', '41', '39'), 2),
 (('36', '39', '38', '41'), 2),
 (('36', '39', '41', '38'), 2),
 (('36', '41', '38', '39'), 2),
 (('36', '41', '39', '38'), 2),
 (('36', '48'), 3),
 (('36', '38', '48'), 3),
 (('36', '39', '48'), 2),
 (('36', '41', '48'), 2),
 (('36', '48', '38'), 3),
 (('36', '48', '39'), 2),
 (('36', '48', '41'), 2),
 (('36', '38', '39', '48'), 2),
 (('36', '38', '41', '48'), 2),
 (('36', '38', '48', '39'), 2),
 (('36', '38', '48', '41'), 2),
 (('36', '39', '38', '48'), 2),
 (('36', '39', '48', '38'), 2),
 (('36', '41', '38', '48'), 2),
 (('36', '41', '48', '38'), 2),
 (('36', '48', '38

In [312]:
item_pairs = frequent_itemsets.cartesian(frequent_itemsets).filter(lambda x: x[0][0] != x[1][0])
sorted(item_pairs.collect(), key=lambda x: int(x[0][0][0]))

[((('32',), 5), (('36',), 6)),
 ((('32',), 5), (('38',), 12)),
 ((('32',), 5), (('39',), 32)),
 ((('32',), 5), (('41',), 11)),
 ((('32',), 5), (('47',), 2)),
 ((('32',), 5), (('65',), 3)),
 ((('32',), 5), (('78',), 2)),
 ((('32',), 5), (('79',), 3)),
 ((('32',), 5), (('89',), 2)),
 ((('32',), 5), (('186',), 2)),
 ((('32',), 5), (('193',), 2)),
 ((('32',), 5), (('225',), 2)),
 ((('32',), 5), (('242',), 3)),
 ((('32',), 5), (('36', '38'), 6)),
 ((('32',), 5), (('36', '39'), 5)),
 ((('32',), 5), (('36', '41'), 3)),
 ((('32',), 5), (('38', '36'), 6)),
 ((('32',), 5), (('38', '39'), 11)),
 ((('32',), 5), (('38', '41'), 4)),
 ((('32',), 5), (('39', '36'), 5)),
 ((('32',), 5), (('39', '38'), 11)),
 ((('32',), 5), (('39', '41'), 8)),
 ((('32',), 5), (('41', '36'), 3)),
 ((('32',), 5), (('41', '38'), 4)),
 ((('32',), 5), (('41', '39'), 8)),
 ((('32',), 5), (('38', '47'), 2)),
 ((('32',), 5), (('39', '47'), 2)),
 ((('32',), 5), (('47', '38'), 2)),
 ((('32',), 5), (('47', '39'), 2)),
 ((('32',), 

In [303]:
getConfidence(('39',), ('41',))

0.25

In [313]:
rdd_join = item_pairs.map(lambda x: (x[0][0] + x[1][0], (x[0], x[1]))).join(frequent_itemsets)

sorted(rdd_join.collect(), key=lambda x: int(x[0][0]))

[(('32', '41'), (((('32',), 5), (('41',), 11)), 2)),
 (('32', '39', '48'), (((('32',), 5), (('39', '48'), 18)), 2)),
 (('32', '39', '48'), (((('32', '39'), 2), (('48',), 23)), 2)),
 (('32', '48', '39'), (((('32',), 5), (('48', '39'), 18)), 2)),
 (('32', '48', '39'), (((('32', '48'), 2), (('39',), 32)), 2)),
 (('32', '39'), (((('32',), 5), (('39',), 32)), 2)),
 (('32', '48'), (((('32',), 5), (('48',), 23)), 2)),
 (('36', '38'), (((('36',), 6), (('38',), 12)), 6)),
 (('36', '38', '41'), (((('36',), 6), (('38', '41'), 4)), 3)),
 (('36', '38', '41'), (((('36', '38'), 6), (('41',), 11)), 3)),
 (('36', '41', '39'), (((('36',), 6), (('41', '39'), 8)), 2)),
 (('36', '41', '39'), (((('36', '41'), 3), (('39',), 32)), 2)),
 (('36', '38', '39', '41'), (((('36',), 6), (('38', '39', '41'), 3)), 2)),
 (('36', '38', '39', '41'), (((('36', '38'), 6), (('39', '41'), 8)), 2)),
 (('36', '38', '39', '41'), (((('36', '38', '39'), 5), (('41',), 11)), 2)),
 (('36', '38', '41', '39'), (((('36',), 6), (('38', '

In [314]:
rdd_map = rdd_join.map(lambda x: (x[1][0][0][0], x[1][0][1][0], x[1][1]/x[1][0][0][1]))

sorted(rdd_map.collect(), key=lambda x: x[2])

[(('39',), ('32',), 0.0625),
 (('39',), ('47',), 0.0625),
 (('39',), ('78',), 0.0625),
 (('39',), ('89',), 0.0625),
 (('39',), ('36', '41'), 0.0625),
 (('39',), ('41', '36'), 0.0625),
 (('39',), ('47', '38'), 0.0625),
 (('39',), ('146', '48'), 0.0625),
 (('39',), ('48', '256'), 0.0625),
 (('39',), ('36', '41', '38'), 0.0625),
 (('39',), ('146',), 0.0625),
 (('39',), ('48', '47'), 0.0625),
 (('39',), ('56', '38'), 0.0625),
 (('39',), ('36', '48'), 0.0625),
 (('39',), ('48', '89'), 0.0625),
 (('39',), ('89', '48'), 0.0625),
 (('39',), ('32', '48'), 0.0625),
 (('39',), ('48', '38', '47'), 0.0625),
 (('39',), ('48', '47', '38'), 0.0625),
 (('39',), ('36', '38', '48'), 0.0625),
 (('39',), ('48', '38', '36'), 0.0625),
 (('39',), ('256', '48'), 0.0625),
 (('39',), ('65',), 0.0625),
 (('39',), ('38', '47'), 0.0625),
 (('39',), ('38', '36', '41'), 0.0625),
 (('39',), ('41', '36', '38'), 0.0625),
 (('39',), ('41', '38', '36'), 0.0625),
 (('39',), ('48', '32'), 0.0625),
 (('39',), ('48', '36'), 0

In [335]:
rdd_result = rdd_map.map(lambda x: ((x[0], x[1]), x[2]))\
                    .reduceByKey(lambda x, y: x)\
                    .map(lambda x: (x[0][0], x[0][1], x[1]))\
                    .filter(lambda x: x[2]>=0.7)\
                    .filter(lambda x: tuple(sorted(x[0])) == x[0] and tuple(sorted(x[1])) == x[1])
sorted(rdd_result.collect(), key=lambda x: x[2])

[(('41',), ('39',), 0.7272727272727273),
 (('38', '41'), ('36',), 0.75),
 (('38', '41'), ('39',), 0.75),
 (('48',), ('39',), 0.782608695652174),
 (('36',), ('38', '39'), 0.8333333333333334),
 (('36', '38'), ('39',), 0.8333333333333334),
 (('41', '48'), ('39',), 0.8333333333333334),
 (('36',), ('39',), 0.8333333333333334),
 (('38', '48'), ('39',), 0.8571428571428571),
 (('38',), ('39',), 0.9166666666666666),
 (('39', '47'), ('38',), 1.0),
 (('47',), ('38', '39'), 1.0),
 (('89',), ('39',), 1.0),
 (('230',), ('48',), 1.0),
 (('146', '39'), ('48',), 1.0),
 (('36', '39', '41'), ('38',), 1.0),
 (('256', '48'), ('39',), 1.0),
 (('256',), ('48',), 1.0),
 (('38', '56'), ('39',), 1.0),
 (('36', '39', '48'), ('38',), 1.0),
 (('39', '89'), ('48',), 1.0),
 (('89',), ('48',), 1.0),
 (('32', '48'), ('39',), 1.0),
 (('47', '48'), ('38',), 1.0),
 (('47', '48'), ('39',), 1.0),
 (('47',), ('38', '39', '48'), 1.0),
 (('38', '47'), ('48',), 1.0),
 (('36', '48'), ('38',), 1.0),
 (('36', '41', '48'), ('38',)

In [336]:
len(rdd_result.collect())

68

In [339]:
# check the result:
# the item must exist in frequent itemset, if not it would allert error

from_ = ('48', '89')
to_ = ('39',)

# expected result:
print('expected: ', getConfidence(from_, to_,))

# received result:
print('received:', getConfidenceFromResult(rdd_map, from_, to_))

expected:  1.0
received: [(('48', '89'), ('39',), 1.0)]
